# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer() 
    tokenizer.fit_on_texts(x)
    
    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    
    return pad_sequences(x, maxlen=length, padding='post', truncating='pre')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [6]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [7]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [8]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    print(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    input_layer = Input(shape = input_shape[1:])
    recurrent_layer = GRU(output_sequence_length, return_sequences=True)(input_layer)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(recurrent_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    learning_rate = 0.01  
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21, 1) 21 199 344
(137861, 21, 1) 21 200 345
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 60s - loss: 2.7978 - acc: 0.4661 - val_loss: 2.1878 - val_acc: 0.5144
Epoch 2/10
110288/110288 [==============================] - 59s - loss: 2.0307 - acc: 0.5334 - val_loss: 1.9005 - val_acc: 0.5549
Epoch 3/10
110288/110288 [==============================] - 59s - loss: 1.8102 - acc: 0.5657 - val_loss: 1.7335 - val_acc: 0.5778
Epoch 4/10
110288/110288 [==============================] - 60s - loss: 1.6774 - acc: 0.5826 - val_loss: 1.6242 - val_acc: 0.5885
Epoch 5/10
110288/110288 [==============================] - 62s - loss: 1.5823 - acc: 0.5920 - val_loss: 1.5477 - val_acc: 0.5949
Epoch 6/10
110288/110288 [==============================] - 64s - loss: 1.5220 - acc: 0.6000 - val_loss: 1.4994 - val_acc: 0.6095
Epoch 7/10
110288/110288 [==============================] - 59s - loss: 1.4775 - acc: 0.6072 - val_loss: 1.4575 - v

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [9]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    print(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    input_layer = Input(shape = (input_shape[1],))
    embedding_layer = Embedding(english_vocab_size, 100)(input_layer)
    recurrent_layer = GRU(output_sequence_length, return_sequences=True)(embedding_layer)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(recurrent_layer)
    model = Model(inputs=input_layer, outputs=output_layer)

    learning_rate = 0.01  
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21) 21 199 344
(137861, 21) 21 200 345
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 64s - loss: 2.5250 - acc: 0.5184 - val_loss: 1.3493 - val_acc: 0.6595
Epoch 2/10
110288/110288 [==============================] - 64s - loss: 1.0783 - acc: 0.7044 - val_loss: 0.8900 - val_acc: 0.7487
Epoch 3/10
110288/110288 [==============================] - 64s - loss: 0.7911 - acc: 0.7783 - val_loss: 0.7191 - val_acc: 0.7949
Epoch 4/10
110288/110288 [==============================] - 64s - loss: 0.6749 - acc: 0.8066 - val_loss: 0.6373 - val_acc: 0.8162
Epoch 5/10
110288/110288 [==============================] - 64s - loss: 0.6132 - acc: 0.8209 - val_loss: 0.5930 - val_acc: 0.8238
Epoch 6/10
110288/110288 [==============================] - 64s - loss: 0.5723 - acc: 0.8309 - val_loss: 0.5539 - val_acc: 0.8355
Epoch 7/10
110288/110288 [==============================] - 64s - loss: 0.5424 - acc: 0.8390 - val_loss: 0.5296 - val_acc

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [9]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    print(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    input_layer = Input(shape = input_shape[1:])
    recurrent_layer = Bidirectional(GRU(output_sequence_length, return_sequences=True))(input_layer)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(recurrent_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    learning_rate = 0.01  
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21, 1) 21 199 344
(137861, 21, 1) 21 200 345
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s - loss: 2.6110 - acc: 0.4865 - val_loss: 1.7941 - val_acc: 0.5615
Epoch 2/10
110288/110288 [==============================] - 8s - loss: 1.5621 - acc: 0.5948 - val_loss: 1.4324 - val_acc: 0.6171
Epoch 3/10
110288/110288 [==============================] - 8s - loss: 1.3713 - acc: 0.6295 - val_loss: 1.3201 - val_acc: 0.6362
Epoch 4/10
110288/110288 [==============================] - 8s - loss: 1.2865 - acc: 0.6378 - val_loss: 1.2548 - val_acc: 0.6408
Epoch 5/10
110288/110288 [==============================] - 8s - loss: 1.2340 - acc: 0.6436 - val_loss: 1.2127 - val_acc: 0.6468
Epoch 6/10
110288/110288 [==============================] - 8s - loss: 1.1975 - acc: 0.6487 - val_loss: 1.1838 - val_acc: 0.6511
Epoch 7/10
110288/110288 [==============================] - 8s - loss: 1.1716 - acc: 0.6525 - val_loss: 1.1582 - val_acc

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [11]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    print(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    input_layer = Input(shape = input_shape[1:])
    recurrent_layer = GRU(input_shape[1], return_sequences=False)(input_layer)
    input_layer_2 = RepeatVector(output_sequence_length)(recurrent_layer)
    recurrent_layer_2 = GRU(output_sequence_length, return_sequences=True)(input_layer_2)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(recurrent_layer_2)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    learning_rate = 0.01  
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
tmp_x = preproc_english_sentences
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-1], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 15, 1) 21 199 344
(137861, 15, 1) 21 200 345
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s - loss: 3.0970 - acc: 0.4018 - val_loss: 2.5759 - val_acc: 0.4659
Epoch 2/10
110288/110288 [==============================] - 7s - loss: 2.4867 - acc: 0.4739 - val_loss: 2.3458 - val_acc: 0.4899
Epoch 3/10
110288/110288 [==============================] - 8s - loss: 2.2660 - acc: 0.4998 - val_loss: 2.2033 - val_acc: 0.5067
Epoch 4/10
110288/110288 [==============================] - 8s - loss: 2.1715 - acc: 0.5080 - val_loss: 2.1382 - val_acc: 0.5138
Epoch 5/10
110288/110288 [==============================] - 7s - loss: 2.1050 - acc: 0.5218 - val_loss: 2.0676 - val_acc: 0.5234
Epoch 6/10
110288/110288 [==============================] - 7s - loss: 2.0323 - acc: 0.5326 - val_loss: 1.9949 - val_acc: 0.5388
Epoch 7/10
110288/110288 [==============================] - 8s - loss: 1.9557 - acc: 0.5370 - val_loss: 1.8754 - val_acc:

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [20]:
# Comment for the reviewer: 
# I created this model_final with the encoder/decoder model because it was the only way I could pass the
# test. The test is passing different lenghts for input and output (15, 21 respectevely)
# Said that, this model does not perform as well as a model where I add Embeddings and Bidirectional together
# Is it OK that this model isn't as good as the one I choose? The encoder/decoder model seem to be more popular
# So I created model_final_2 and use it in the final prediction.

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    print(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    input_layer = Input(shape = (input_shape[1],))
    embedding_layer = Embedding(english_vocab_size, 100)(input_layer)
    recurrent_layer_1_1 = Bidirectional(GRU(output_sequence_length, return_sequences=False))(embedding_layer)
    input_layer_2 = RepeatVector(output_sequence_length)(recurrent_layer_1_1)
    recurrent_layer_2 = Bidirectional(GRU(output_sequence_length, return_sequences=True))(input_layer_2)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(recurrent_layer_2)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    learning_rate = 0.01  
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_model_final(model_final)
print('Final Model Loaded')

tmp_x = preproc_english_sentences
#tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-1], 1))

# Train the neural network
final_rnn_model = model_final(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=50, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 15) 21 199 344
Final Model Loaded
(137861, 15) 21 200 345
Train on 110288 samples, validate on 27573 samples
Epoch 1/50
110288/110288 [==============================] - 15s - loss: 2.9200 - acc: 0.4345 - val_loss: 2.4837 - val_acc: 0.4796
Epoch 2/50
110288/110288 [==============================] - 13s - loss: 2.2940 - acc: 0.4884 - val_loss: 2.1286 - val_acc: 0.4857
Epoch 3/50
110288/110288 [==============================] - 13s - loss: 2.0114 - acc: 0.5139 - val_loss: 1.9103 - val_acc: 0.5343
Epoch 4/50
110288/110288 [==============================] - 13s - loss: 1.8549 - acc: 0.5339 - val_loss: 1.8138 - val_acc: 0.5314
Epoch 5/50
110288/110288 [==============================] - 13s - loss: 1.7468 - acc: 0.5548 - val_loss: 1.6857 - val_acc: 0.5747
Epoch 6/50
110288/110288 [==============================] - 13s - loss: 1.6439 - acc: 0.5840 - val_loss: 1.6171 - val_acc: 0.5822
Epoch 7/50
110288/110288 [==============================] - 13s - loss: 1.5465 - acc: 0.6038 - val_los

In [24]:
# This is the final model I really used for the predictions

def model_final_2(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    print(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    input_layer = Input(shape = (input_shape[1],))
    embedding_layer = Embedding(english_vocab_size, 100)(input_layer)
    recurrent_layer = Bidirectional(GRU(output_sequence_length, return_sequences=True))(embedding_layer)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation="softmax"))(recurrent_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    learning_rate = 0.01  
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

print('Final Model Loaded')

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# Train the neural network
final_rnn_model_2 = model_final_2(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model_2.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model_2.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
(137861, 21) 21 200 345
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s - loss: 1.9362 - acc: 0.6059 - val_loss: 0.9179 - val_acc: 0.7491
Epoch 2/10
110288/110288 [==============================] - 8s - loss: 0.7192 - acc: 0.7949 - val_loss: 0.5877 - val_acc: 0.8268
Epoch 3/10
110288/110288 [==============================] - 8s - loss: 0.5241 - acc: 0.8435 - val_loss: 0.4689 - val_acc: 0.8604
Epoch 4/10
110288/110288 [==============================] - 8s - loss: 0.4360 - acc: 0.8698 - val_loss: 0.4033 - val_acc: 0.8813
Epoch 5/10
110288/110288 [==============================] - 8s - loss: 0.3833 - acc: 0.8866 - val_loss: 0.3610 - val_acc: 0.8948
Epoch 6/10
110288/110288 [==============================] - 8s - loss: 0.3503 - acc: 0.8977 - val_loss: 0.3427 - val_acc: 0.9007
Epoch 7/10
110288/110288 [==============================] - 8s - loss: 0.3266 - acc: 0.9056 - val_loss: 0.3164 - val_acc: 0.9098
Ep

## Prediction (IMPLEMENTATION)

In [27]:
# This line at the end of the function make no sense for me. Am I missing anything?
# print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))
# should it be the text?

import numpy as np
from keras.preprocessing.sequence import pad_sequences


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    x = pad(x, y.shape[1])
    
    model = model_final_2(
        x.shape,
        y.shape[1],
        len(x_tk.word_index)+1,
        len(y_tk.word_index)+1)
    
    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    # Is this line correct?! 
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

(137861, 21) 21 200 345
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 11s - loss: 1.9974 - acc: 0.5940 - val_loss: 0.9457 - val_acc: 0.7256
Epoch 2/20
110288/110288 [==============================] - 9s - loss: 0.7574 - acc: 0.7721 - val_loss: 0.6045 - val_acc: 0.8230
Epoch 3/20
110288/110288 [==============================] - 9s - loss: 0.5320 - acc: 0.8422 - val_loss: 0.4767 - val_acc: 0.8578
Epoch 4/20
110288/110288 [==============================] - 9s - loss: 0.4432 - acc: 0.8674 - val_loss: 0.4179 - val_acc: 0.8743
Epoch 5/20
110288/110288 [==============================] - 9s - loss: 0.3953 - acc: 0.8818 - val_loss: 0.3806 - val_acc: 0.8869
Epoch 6/20
110288/110288 [==============================] - 9s - loss: 0.3668 - acc: 0.8903 - val_loss: 0.3498 - val_acc: 0.8955
Epoch 7/20
110288/110288 [==============================] - 9s - loss: 0.3417 - acc: 0.8989 - val_loss: 0.3416 - val_acc: 0.8972
Epoch 8/20
110288/110

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.